In [1]:
DATA_NAME = 'segmentAnything-full' 
TRANSFORM = 'wavelet-diagonal'
CHANNEL = 'gray'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_gray_diagonal_wavelet_full_segmentAnything.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10
2,28734.38,-0.18,-0.02,0.10,0.00,-0.01,0.00,-0.00,-0.00
3,-0.18,6587.25,-0.04,0.02,-0.02,0.00,0.00,0.00,0.00
4,-0.02,-0.04,1419.76,0.02,-0.00,-0.00,-0.00,-0.00,0.00
5,0.10,0.02,0.02,285.85,0.00,-0.00,0.00,0.00,0.00
6,0.00,-0.02,-0.00,0.00,54.70,-0.00,-0.00,0.00,0.00
7,-0.01,0.00,-0.00,-0.00,-0.00,10.18,0.00,0.00,0.00
8,0.00,0.00,-0.00,0.00,-0.00,0.00,1.85,0.00,0.00
9,-0.00,0.00,-0.00,0.00,0.00,0.00,0.00,0.31,-0.00
10,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,0.04


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10
2,1.00000,-0.00001,-0.00000,0.00003,0.00000,-0.00002,0.00001,-0.00001,-0.00000
3,-0.00001,1.00000,-0.00001,0.00001,-0.00003,0.00001,0.00004,0.00000,0.00000
4,-0.00000,-0.00001,1.00000,0.00003,-0.00000,-0.00000,-0.00003,-0.00001,0.00001
5,0.00003,0.00001,0.00003,1.00000,0.00001,-0.00002,0.00005,0.00000,0.00000
6,0.00000,-0.00003,-0.00000,0.00001,1.00000,-0.00001,-0.00001,0.00002,0.00001
7,-0.00002,0.00001,-0.00000,-0.00002,-0.00001,1.00000,0.00002,0.00000,0.00000
8,0.00001,0.00004,-0.00003,0.00005,-0.00001,0.00002,1.00000,0.00003,0.00000
9,-0.00001,0.00000,-0.00001,0.00000,0.00002,0.00000,0.00003,1.00000,-0.00001
10,-0.00000,0.00000,0.00001,0.00000,0.00001,0.00000,0.00000,-0.00001,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.2984893773864544

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[9.18236727e+07 4.74684033e+06 2.16007369e+05 8.51461442e+03
 2.99390879e+02 9.72182104e+00 2.86226921e-01 6.11530594e-03
 2.24492421e-35]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8
0,0.999524,-0.030200,-0.006140,-0.001230,-0.000236,-0.000044,-0.000008,-0.000001,-1.781226e-07
1,0.029978,0.999004,-0.032445,-0.006195,-0.001186,-0.000220,-0.000039,-0.000007,-8.673297e-07
2,0.007058,0.032002,0.998824,-0.035116,-0.006417,-0.001192,-0.000218,-0.000037,-4.697456e-06
3,0.001649,0.007218,0.034598,0.998552,-0.039878,-0.007108,-0.001286,-0.000217,-2.814197e-05
4,0.000379,0.001655,0.007675,0.039222,0.997993,-0.048331,-0.008421,-0.001413,-1.835989e-04
5,0.000088,0.000383,0.001777,0.008840,0.047368,0.996810,-0.062775,-0.010112,-1.311565e-03
6,0.000021,0.000092,0.000430,0.002126,0.011140,0.061175,0.994257,-0.086404,-1.075618e-02
7,0.000005,0.000023,0.000107,0.000532,0.002782,0.014970,0.083485,0.987850,-1.301909e-01
8,0.000001,0.000006,0.000028,0.000140,0.000733,0.003939,0.021665,0.128770,9.914297e-01


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0004757844839239356,
 0.0009960744955993528,
 0.0011755233952881472,
 0.0014482214340280075,
 0.002006514892505029,
 0.0031900238515389923,
 0.005742721536336148,
 0.012149578047423781,
 0.008570279994963625]